In [2]:
import os
import pickle
import torch
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import defaultdict, Counter
from abc import ABC, abstractmethod
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

warnings.filterwarnings('ignore', message='.*sklearn.utils.parallel.delayed.*', category=UserWarning)

In [3]:
class BaseRecommender(ABC):
    def __init__(self, name):
        self.name = name
        self.train_df = None
        
    @abstractmethod
    def fit(self, train_df):
        """Treina o modelo com os dados de treino."""
        pass
    
    @abstractmethod
    def recommend(self, author_id, top_n=10):
        """Retorna uma lista de author_ids recomendados."""
        pass

In [4]:
class TopologyRecommender(BaseRecommender):
    def __init__(self):
        super().__init__("Topology (Graph Coauthor)")
        self.graph = defaultdict(set)
        self.popular_authors = []
        
    def fit(self, train_df):
        self.train_df = train_df
        print(f"[{self.name}] Construindo grafo...")
        
        # Construção do Grafo
        for _, group in train_df.groupby('work_id'):
            authors = group['author_id'].tolist()
            if len(authors) > 1:
                for u, v in itertools.combinations(authors, 2):
                    self.graph[u].add(v)
        
        # Cálculo de Popularidade (para fallback)
        popularity_counter = Counter()
        for author, neighbors in self.graph.items():
            popularity_counter[author] = len(neighbors)
        self.popular_authors = [auth for auth, _ in popularity_counter.most_common()]
        print(f"[{self.name}] Grafo construído com {len(self.graph)} autores.")

    def recommend(self, author_id, top_n=10):
        recommendations = []
        current_coauthors = self.graph.get(author_id, set())
        
        # Determinar se top_n é negativo (modo mínimo)
        is_minimum_mode = top_n < 0
        target_n = abs(top_n) if is_minimum_mode else top_n
        
        # Lógica de Amigos em Comum (2 hops)
        if author_id in self.graph:
            candidates = []
            for neighbor in current_coauthors:
                neighbors_of_neighbor = self.graph.get(neighbor, set())
                for candidate in neighbors_of_neighbor:
                    if candidate != author_id and candidate not in current_coauthors:
                        candidates.append(candidate)
            
            # Se modo mínimo, pegar todos os amigos em comum
            # Se modo normal, limitar a top_n
            if is_minimum_mode:
                recommendations = [c[0] for c in Counter(candidates).most_common()]
            else:
                recommendations = [c[0] for c in Counter(candidates).most_common(top_n)]
        
        # Fallback: Populares
        # Se modo mínimo, garantir pelo menos target_n recomendações
        # Se modo normal, completar até top_n
        if len(recommendations) < target_n:
            for pop in self.popular_authors:
                if pop != author_id and pop not in recommendations and pop not in current_coauthors:
                    recommendations.append(pop)
                    if len(recommendations) >= target_n:
                        break
        
        # Se modo normal, limitar a top_n
        # Se modo mínimo, retornar todas (já garantimos pelo menos target_n)
        if not is_minimum_mode:
            return recommendations[:top_n]
        else:
            return recommendations


In [5]:
database_path = 'database_50k'
authors_df = pd.read_csv(f'{database_path}/authorships.csv')
works_df = pd.read_csv(f'{database_path}/works.csv')

merged_df = authors_df.merge(
    works_df[['id', 'publication_date', 'title', 'abstract', 'language']], 
    left_on='work_id', right_on='id'
)
merged_df['publication_date'] = pd.to_datetime(merged_df['publication_date'], errors='coerce')
merged_df = merged_df.dropna(subset=['publication_date', 'author_id', 'title', 'abstract', 'language']).drop(columns=['id'])
merged_df = merged_df[merged_df['language'] == 'en']

# Divisão Temporal: 80% Treino, 20% Teste
print("Dividindo dados temporalmente (80/20)...")
unique_works = merged_df[['work_id', 'publication_date']].drop_duplicates().sort_values('publication_date')
total_works = len(unique_works)

split_train = int(total_works * 0.8)

train_work_ids = set(unique_works.iloc[:split_train]['work_id'])
test_work_ids = set(unique_works.iloc[split_train:]['work_id'])

train_df = merged_df[merged_df['work_id'].isin(train_work_ids)]
test_df = merged_df[merged_df['work_id'].isin(test_work_ids)]

print(f"\n=== RESUMO DO SPLIT ===")
print(f"Trabalhos no Treino: {len(train_work_ids)}")
print(f"Trabalhos no Teste: {len(test_work_ids)}")
print(f"Total de Autores no Treino: {len(set(train_df['author_id']))}")
print(f"Total de Autores no Teste: {len(set(test_df['author_id']))}")

def build_graph(df):
    graph = defaultdict(set)
    for _, group in df.groupby('work_id'):
        authors = group['author_id'].tolist()
        
        if len(authors) > 1:
            for u, v in itertools.combinations(authors, 2):
                graph[u].add(v)

    return graph

# Construir grafos para avaliação
train_graph = build_graph(train_df)
test_graph_raw = build_graph(test_df)

# Ground truth para teste: novos links que apareceram no teste mas não no treino
test_ground_truth = defaultdict(set)
for author, coauthors in test_graph_raw.items():
    future_coauthors = coauthors
    past_coauthors = train_graph.get(author, set())
    new_links = future_coauthors - past_coauthors
    
    if new_links:
        test_ground_truth[author] = new_links

print(f"\nAutores alvo no teste (com novos links): {len(test_ground_truth)}")

C:\Users\BSBCo\AppData\Local\Temp\ipykernel_24416\2237052165.py:3: DtypeWarning: Columns (0: is_retracted) have mixed types. Specify dtype option on import or set low_memory=False.
  works_df = pd.read_csv(f'{database_path}/works.csv')


Dividindo dados temporalmente (80/20)...

=== RESUMO DO SPLIT ===
Trabalhos no Treino: 19916
Trabalhos no Teste: 4980
Total de Autores no Treino: 25262
Total de Autores no Teste: 11441

Autores alvo no teste (com novos links): 10007


In [10]:
# Criação do dataset apenas com pares que são coautores
# Dataset terá: author_a, author_b, count_coauthors, common_coauthors

print("Construindo dicionário de contagem de coautorias...")
coauthor_count = {}

# Construir grafo de coautores para calcular autores em comum
print("Construindo grafo de coautores...")
coauthor_graph = defaultdict(set)

for work_id, group in tqdm(train_df.groupby("work_id"), desc="Processando trabalhos"):
    authors = group["author_id"].tolist()
    if len(authors) > 1:
        # Para cada par de autores no trabalho, incrementar o contador
        for a, b in itertools.combinations(authors, 2):
            # Usar min/max ao invés de sorted (mais rápido)
            pair = (min(a, b), max(a, b))
            coauthor_count[pair] = coauthor_count.get(pair, 0) + 1
            # Construir grafo bidirecional
            coauthor_graph[a].add(b)
            coauthor_graph[b].add(a)

print(f"Total de pares que são coautores: {len(coauthor_count):,}")

# Criar dataset apenas com pares que são coautores
print("\nCriando dataset com pares de coautores e calculando autores em comum...")
rows = []

for (author_a, author_b), count in tqdm(
    coauthor_count.items(), desc="Processando pares"
):
    # Calcular quantidade de autores em comum
    coauthors_a = coauthor_graph.get(author_a, set())
    coauthors_b = coauthor_graph.get(author_b, set())
    # Interseção: autores que são coautores de ambos (excluindo o próprio par)
    common_coauthors = (coauthors_a & coauthors_b) - {author_a, author_b}
    common_count = len(common_coauthors)
    
    rows.append({
        "author_a": author_a, 
        "author_b": author_b, 
        "count_coauthors": count,
        "common_coauthors": common_count
    })

dataset_df = pd.DataFrame(rows)
print(f"\nDataset criado! Total de linhas: {len(dataset_df):,}")
display(dataset_df.head(10))

Construindo dicionário de contagem de coautorias...
Construindo grafo de coautores...


Processando trabalhos: 100%|██████████| 19916/19916 [00:03<00:00, 6045.58it/s]


Total de pares que são coautores: 217,493

Criando dataset com pares de coautores e calculando autores em comum...


Processando pares: 100%|██████████| 217493/217493 [00:01<00:00, 216307.88it/s]



Dataset criado! Total de linhas: 217,493


,author_a,author_b,count_coauthors,common_coauthors
0,https://openalex.org/A5076219710,https://openalex.org/A5103241656,1,4
1,https://openalex.org/A5035579644,https://openalex.org/A5103241656,2,6
2,https://openalex.org/A5069881493,https://openalex.org/A5103241656,2,6
3,https://openalex.org/A5006873642,https://openalex.org/A5103241656,2,6
4,https://openalex.org/A5103241656,https://openalex.org/A5110165399,1,4
5,https://openalex.org/A5035579644,https://openalex.org/A5076219710,1,4
6,https://openalex.org/A5069881493,https://openalex.org/A5076219710,1,4
7,https://openalex.org/A5006873642,https://openalex.org/A5076219710,1,4
8,https://openalex.org/A5076219710,https://openalex.org/A5110165399,1,4
9,https://openalex.org/A5035579644,https://openalex.org/A5069881493,3,8


In [9]:
# Buscar todas as relações (coautorias) do autor https://openalex.org/A5076219710
autor_id = "https://openalex.org/A5076219710"

relacoes_autor = dataset_df[
    (dataset_df['author_a'] == autor_id) | (dataset_df['author_b'] == autor_id)
]

print(f"Relações (coautorias) do autor {autor_id}:")
display(relacoes_autor)


Relações (coautorias) do autor https://openalex.org/A5076219710:


,author_a,author_b,count_coauthors
0,https://openalex.org/A5076219710,https://openalex.org/A5103241656,1
5,https://openalex.org/A5035579644,https://openalex.org/A5076219710,1
6,https://openalex.org/A5069881493,https://openalex.org/A5076219710,1
7,https://openalex.org/A5006873642,https://openalex.org/A5076219710,1
8,https://openalex.org/A5076219710,https://openalex.org/A5110165399,1
